In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
number_of_features = 28 * 28
w = tf.Variable(tf.random_normal([number_of_features], name="weights"))
b = tf.Variable(0.0, name="bias")
x = tf.placeholder(tf.float32, shape=(None, number_of_features), name="input")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
output = logistic_regression(w, b, x)

error = output - y

mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.2)
training_op = optimizer.minimize(mse)
correctness = tf.reduce_mean(tf.cast(tf.abs(error) < 0.1, tf.float32)) * 100

In [ ]:
epochs = 10
batch_size = 50
n_batches = int(np.ceil(mnist.train.num_examples // batch_size))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/mnist_model.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', mse)
correctness_summary = tf.summary.scalar("correctness", correctness)
summary_op = tf.summary.merge([mse_summary, correctness_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    #saver.restore(sess, interim_checkpoint_path)
    
    for epoch in range(epochs):
        for batch_index in range(n_batches):
            batch_X = moon_X[batch_index * batch_size:(batch_index + 1) * batch_size]
            #print("batch_X shape:", batch_X.shape)
            batch_y = moon_y[batch_index * batch_size:(batch_index + 1) * batch_size].reshape((-1, 1))
            #print("batch_Y shape:", batch_y.shape)
            if batch_index % 10 == 0:
                summary_str = summary_op.eval(feed_dict={x: batch_X, y: batch_y})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            t, m = sess.run([training_op, mse], feed_dict={x: batch_X, y: batch_y})
            
            #print("Batch performance: ", batch_correctness)
            if batch_index == 0: print("Mean squared error:", m)
        if epoch % 100 == 0:
            save_path = saver.save(sess, interim_checkpoint_path)

    save_path = saver.save(sess, "./checkpoints/mnist_model_final.ckpt")